# Sklearn

In [0]:
# Run Tensorboard in the background
LOGDIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOGDIR)
)
# Use ngrok to tunnel traffic to localhost
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
# Retrieve public url
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-07-04 05:50:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.145.109, 54.152.127.232, 52.45.111.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.145.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17556757 (17M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  16.74M  20.9MB/s    in 0.8s    

2019-07-04 05:51:05 (20.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [17556757/17556757]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://63651f8c.ngrok.io


## sklearn.model_selection

документация: http://scikit-learn.org/stable/modules/cross_validation.html

In [0]:
from sklearn import model_selection, datasets

import numpy as np

### Разовое разбиение данных на обучение и тест с помощью train_test_split

In [0]:
iris = datasets.load_iris()

In [0]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3)

In [0]:
#убедимся, что тестовая выборка действительно составляет 0.3 от всех данных
float(len(test_labels))/len(iris.data)

0.3

In [0]:
print 'Размер обучающей выборки: {} объектов \nРазмер тестовой выборки: {} объектов'.format(len(train_data),
                                                                                            len(test_data))

Размер обучающей выборки: 105 объектов 
Размер тестовой выборки: 45 объектов


In [0]:
print 'Обучающая выборка:\n', train_data[:5]
print '\n'
print 'Тестовая выборка:\n', test_data[:5]

Обучающая выборка:
[[6.3 3.3 6.  2.5]
 [5.4 3.9 1.7 0.4]
 [4.5 2.3 1.3 0.3]
 [5.4 3.7 1.5 0.2]
 [4.8 3.1 1.6 0.2]]


Тестовая выборка:
[[7.7 2.6 6.9 2.3]
 [6.4 3.2 5.3 2.3]
 [4.7 3.2 1.6 0.2]
 [5.5 2.4 3.7 1. ]
 [7.2 3.  5.8 1.6]]


In [0]:
print 'Метки классов на обучающей выборке:\n', train_labels
print '\n'
print 'Метки классов на тестовой выборке:\n', test_labels

### Стратегии проведения кросс-валидации

In [0]:
#сгенерируем короткое подобие датасета, где элементы совпадают с порядковым номером
X = range(0,10)

#### KFold

In [0]:
kf = model_selection.KFold(n_splits = 5)
for train_indices, test_indices in kf.split(X):
    print train_indices, test_indices

[2 3 4 5 6 7 8 9] [0 1]
[0 1 4 5 6 7 8 9] [2 3]
[0 1 2 3 6 7 8 9] [4 5]
[0 1 2 3 4 5 8 9] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


In [0]:
kf = model_selection.KFold(n_splits = 2, shuffle = True)
for train_indices, test_indices in kf.split(X):
    print train_indices, test_indices

[2 4 6 7 8] [0 1 3 5 9]
[0 1 3 5 9] [2 4 6 7 8]


In [0]:
kf = model_selection.KFold(n_splits = 2, shuffle = True, random_state = 1)
for train_indices, test_indices in kf.split(X):
    print train_indices, test_indices

[1 3 5 7 8] [0 2 4 6 9]
[0 2 4 6 9] [1 3 5 7 8]


#### StratifiedKFold

In [0]:
y = np.array([0] * 5 + [1] * 5)
print y

skf = model_selection.StratifiedKFold(n_splits = 2, shuffle = True, random_state = 0)
for train_indices, test_indices in skf.split(X, y):
    print train_indices, test_indices

[0 0 0 0 0 1 1 1 1 1]
[3 4 8 9] [0 1 2 5 6 7]
[0 1 2 5 6 7] [3 4 8 9]


In [0]:
target = np.array([0, 1] * 5)
print target

skf = model_selection.StratifiedKFold(n_splits = 2,shuffle = True)
for train_indices, test_indices in skf.split(X, target):
    print train_indices, test_indices

[0 1 0 1 0 1 0 1 0 1]
[1 2 6 7] [0 3 4 5 8 9]
[0 3 4 5 8 9] [1 2 6 7]


#### ShuffleSplit

In [0]:
ss = model_selection.ShuffleSplit(n_splits = 10, test_size = 0.2)

for train_indices, test_indices in ss.split(X):
    print train_indices, test_indices

[4 1 3 0 8 7 5 2] [6 9]
[6 7 9 8 0 3 2 4] [1 5]
[3 1 5 4 6 0 8 9] [2 7]
[8 9 0 7 1 3 4 2] [6 5]
[8 1 2 5 7 0 9 6] [3 4]
[9 1 3 7 8 0 5 4] [2 6]
[1 3 5 0 9 7 2 8] [4 6]
[5 6 4 0 3 8 7 9] [1 2]
[7 9 0 8 1 2 6 3] [5 4]
[7 6 3 2 9 1 0 4] [8 5]


#### StratifiedShuffleSplit

In [0]:
target = np.array([0] * 5 + [1] * 5)
print target

sss = model_selection.StratifiedShuffleSplit(n_splits = 4, test_size = 0.2)
for train_indices, test_indices in sss.split(X, target):
    print train_indices, test_indices

[0 0 0 0 0 1 1 1 1 1]
[2 1 9 5 4 7 6 0] [3 8]
[3 4 6 5 7 0 2 9] [1 8]
[7 4 2 1 3 6 5 8] [9 0]
[6 8 3 1 7 9 2 4] [5 0]


#### Leave-One-Out

In [0]:
loo = model_selection.LeaveOneOut()

for train_indices, test_index in loo.split(X):
    print train_indices, test_index

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


Больше стратегий проведения кросс-валидации доступно здесь: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators